In [6]:
import sqlite3
import pandas as pd

In [7]:
# Loading file 
df = pd.read_csv('olist_customers_dataset.csv')
df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [8]:
# Create SQLite database
conn = sqlite3.connect(':memory:')
# Load DataFrame into SQLite table
df.to_sql('olist_customers', conn, index=False, if_exists='replace')

99441

In [11]:
# Sample table for joins
data = {
    'order_id': ['O1', 'O2', 'O3', 'O4'],
    'customer_id': ['C1', 'C2', 'C1', 'C3'],
    'order_status': ['delivered', 'shipped', 'cancelled', 'delivered'],
    'total_amount': [100, 200, 150, 300]
}

df_orders = pd.DataFrame(data)
df_orders.to_sql('orders', conn, index=False, if_exists='replace')

4

In [12]:
#SELECT, WHERE, ORDER BY, GROUP BY
pd.read_sql_query("""
SELECT customer_state, COUNT(*) AS num_customers
FROM olist_customers
WHERE customer_state != ''
GROUP BY customer_state
ORDER BY num_customers DESC;
""", conn)

,customer_state,num_customers
0,SP,41746
1,RJ,12852
2,MG,11635
3,RS,5466
4,PR,5045
5,SC,3637
6,BA,3380
7,DF,2140
8,ES,2033
9,GO,2020


In [14]:
#INNER JOIN
pd.read_sql_query("""
SELECT o.order_id, c.customer_city, c.customer_state
FROM orders o
INNER JOIN olist_customers c
ON o.customer_id = c.customer_id;
""", conn)
#LEFT JOIN
pd.read_sql_query("""
SELECT c.customer_id, c.customer_city, o.order_id
FROM olist_customers c
LEFT JOIN orders o
ON c.customer_id = o.customer_id;
""", conn)

,customer_id,customer_city,order_id
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,None
1,18955e83d337fd6b2def6b18a428ac77,sao bernardo do campo,None
2,4e7b3e00288586ebd08712fdd0374a03,sao paulo,None
3,b2b6027bc5c5109e529d4dc6358b12c3,mogi das cruzes,None
4,4f2d8ab171c80ec8364f7c12e35b23ad,campinas,None
...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,sao paulo,None
99437,e7b71a9017aa05c9a7fd292d714858e8,taboao da serra,None
99438,5e28dfe12db7fb50a4b2f691faecea5e,fortaleza,None
99439,56b18e2166679b8a959d72dd06da27f9,canoas,None


In [15]:
#Subquery to return all customers from cities having more than one customer
pd.read_sql_query("""
SELECT *
FROM olist_customers
WHERE customer_city IN (
  SELECT customer_city
  FROM olist_customers
  GROUP BY customer_city
  HAVING COUNT(*) > 1
);
""", conn)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
98292,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
98293,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
98294,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
98295,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [16]:
#d. Aggregate Functions (SUM, AVG)
pd.read_sql_query("""
SELECT o.customer_id, SUM(o.total_amount) AS total_spent, AVG(o.total_amount) AS avg_spent
FROM orders o
GROUP BY o.customer_id;
""", conn)

,customer_id,total_spent,avg_spent
0,C1,250,125.0
1,C2,200,200.0
2,C3,300,300.0


In [17]:
#Creating Views for Analysis
# Create a view for top 5 states by customer count
conn.execute("""
CREATE VIEW top_states AS
SELECT customer_state, COUNT(*) AS num_customers
FROM olist_customers
GROUP BY customer_state
ORDER BY num_customers DESC
LIMIT 5;
""")

# Query the view
pd.read_sql_query("SELECT * FROM top_states;", conn)

,customer_state,num_customers
0,SP,41746
1,RJ,12852
2,MG,11635
3,RS,5466
4,PR,5045


In [18]:
#f. Optimizing Queries with Indexes
# Index for faster JOINs
conn.execute("CREATE INDEX idx_customer_id ON olist_customers(customer_id);")

# Index for faster filtering/grouping
conn.execute("CREATE INDEX idx_customer_state ON olist_customers(customer_state);")
